In [46]:
import ast
import copy
import re

from collections import defaultdict 
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from aocd import get_data, submit
from functools import cmp_to_key
import operator


DAY = 7
YEAR = 2023

current_day is only available in December (EST)


In [2]:
# use test data
raw_test = """input"""

# use real data
raw = get_data(day=DAY, year=YEAR)

print(raw_test)

input


In [5]:
def parse_data(data):
    data = data.split("\n")
    data = [d.split(" ") for d in data]
    # data = [d.split('\n') for d in data.split('\n\n')]
    # data = [np.array(d.split('\n'), dtype=int) for d in data.split('\n\n')]

    return data


dummy = parse_data(raw_test)
real = parse_data(raw)

real

[['3A399', '27'],
 ['46645', '201'],
 ['8A9K4', '40'],
 ['88333', '578'],
 ['55353', '817'],
 ['99Q99', '146'],
 ['553J5', '45'],
 ['AAQ2A', '547'],
 ['TKQ2K', '120'],
 ['57592', '534'],
 ['7Q49K', '229'],
 ['JK949', '744'],
 ['688T6', '657'],
 ['98888', '392'],
 ['4K638', '337'],
 ['J8888', '96'],
 ['Q92KJ', '778'],
 ['48J96', '374'],
 ['955J6', '307'],
 ['62TT6', '896'],
 ['934AK', '776'],
 ['44654', '720'],
 ['AJAAA', '568'],
 ['K6J5Q', '162'],
 ['T29A4', '342'],
 ['46342', '467'],
 ['22227', '694'],
 ['TQ4TT', '468'],
 ['88688', '728'],
 ['88999', '183'],
 ['Q7T6J', '463'],
 ['92T9J', '884'],
 ['KAQQQ', '299'],
 ['3275Q', '913'],
 ['Q5535', '82'],
 ['93A4J', '48'],
 ['J9888', '552'],
 ['2QTQ4', '846'],
 ['44554', '795'],
 ['JJJJJ', '348'],
 ['TT477', '733'],
 ['58883', '460'],
 ['77377', '317'],
 ['357J3', '725'],
 ['T9J4Q', '346'],
 ['88228', '953'],
 ['Q4858', '982'],
 ['5T5TT', '405'],
 ['TTKTT', '322'],
 ['QKKKK', '197'],
 ['A5Q5A', '956'],
 ['2T22A', '962'],
 ['TQJTT', '734'],

# Part 1

In [43]:
cards = ['2','3','4','5','6','7','8','9','T','J','Q','K','A'] #value is determined by index

hand_type = [
    [1,1,1,1,1],
    [1, 1, 1, 2],
    [1, 2, 2],
    [1, 1, 3],
    [2, 3],
    [1, 4],
    [5]
]


def compare(hand1, hand2):
    for i in range(len(hand1)):
        if cards.index(hand1[i]) == cards.index(hand2[i]):
            continue
        elif cards.index(hand1[i]) < cards.index(hand2[i]):
            return -1
        else:
            return 1

compare_cards = cmp_to_key(compare)

#create a new dict to count every card occurrence
# five of a kind - count is 5
# four of a kind - count is 4
# full house - count is 3 & 2
# three of a kind - count is 3
# two pair - count is 2 & 2
# one pair - count is 2
# highest card - count is 1


card_count = dict()
card_bid = dict()
sorted_hand_type = dict()
sorted_hands = list()

for data in real:
    card_count[data[0]] = defaultdict(int)
    card_bid[data[0]] = int(data[1])
    for d in data[0]:
        card_count[data[0]][d] += 1
    
    hand_type_value = hand_type.index(sorted(card_count[data[0]].values()))
    if hand_type_value not in list(sorted_hand_type.keys()):
        sorted_hand_type[hand_type_value] = list()
    sorted_hand_type[hand_type_value].append(data[0])
    
# compare i-th card if the type of the hands are the same
for hand_type_val in sorted(list(sorted_hand_type.keys())):
    sorted_hands += sorted(sorted_hand_type[hand_type_val], key=compare_cards)


# higher rank = higher multiplier of the bid (starts with 1)
total_score = 0
for rank, hand in enumerate(sorted_hands, start=1):
    total_score += (card_bid[hand] * rank)
    
result = total_score
print(result)

255048101


In [35]:
submit(result, part="a", day=DAY, year=YEAR)

That's the right answer!  You are one gold star closer to restoring snow operations. [Continue to Part Two]


<Response [200]>

# Part 2

In [50]:
cards_j = ['J','2','3','4','5','6','7','8','9','T','Q','K','A'] #value is determined by index

def compare(hand1, hand2):
    for i in range(len(hand1)):
        if cards_j.index(hand1[i]) == cards_j.index(hand2[i]):
            continue
        elif cards_j.index(hand1[i]) < cards_j.index(hand2[i]):
            return -1
        else:
            return 1

compare_cards = cmp_to_key(compare)



card_count = dict()
card_bid = dict()
sorted_hand_type = dict()
sorted_hands = list()

for data in real:
    card_count[data[0]] = defaultdict(int)
    card_bid[data[0]] = int(data[1])
    for d in data[0]:
        card_count[data[0]][d] += 1
    
    # adjust counts for jokers
    if 'J' in list(card_count[data[0]].keys()):
        nr_jokers = card_count[data[0]].pop('J', None)
        if nr_jokers == len(data[0]):
            most_of_card = cards_j[-1] #if all cards are jokers change them to the highest valued
        else:
            most_of_card = max(card_count[data[0]], key=card_count[data[0]].get)
        card_count[data[0]][most_of_card] += nr_jokers
        
    hand_type_value = hand_type.index(sorted(card_count[data[0]].values()))
    
    if hand_type_value not in list(sorted_hand_type.keys()):
        sorted_hand_type[hand_type_value] = list()
    sorted_hand_type[hand_type_value].append(data[0])
        
# compare i-th card if the type of the hands are the same
for hand_type_val in sorted(list(sorted_hand_type.keys())):
    sorted_hands += sorted(sorted_hand_type[hand_type_val], key=compare_cards)


# higher rank = higher multiplier of the bid (starts with 1)
total_score = 0
for rank, hand in enumerate(sorted_hands, start=1):
    total_score += (card_bid[hand] * rank)
    
result = total_score
print(result)

253718286


In [51]:
submit(result, part="b", day=DAY, year=YEAR)

That's the right answer!  You are one gold star closer to restoring snow operations.You have completed Day 7! You can [Shareon
  Twitter
Mastodon] this victory or [Return to Your Advent Calendar].


<Response [200]>